In [8]:
!pip install PySastrawi

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import json
import pickle
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
import pandas as pd
from openpyxl import load_workbook
import datetime

In [12]:
# Load the intents file
with open('dataset/nlp_dataset_faq.json') as data_file:
    intents = json.load(data_file)

In [13]:
import nltk

# Download the 'punkt_tab' data package
nltk.download('punkt_tab')

# Pre-processing
words = []
classes = []
documents = []
ignore = ['!', '"', "'", '(', ')', ',', '-', '.', ':', ';', '?', '[', ']', '_',
          'adalah', 'akan', 'aku', 'anda', 'atau', 'bisa', 'dalam', 'dan', 'dapat',
          'dari', 'dengan', 'di', 'dia', 'harus', 'ini', 'itu', 'jika', 'kami', 'kamu', 'ke',
          'kita', 'mereka', 'oleh', 'pada', 'saya', 'sebuah', 'sedang', 'sementara', 'tanpa',
          'tapi', 'telah', 'untuk', 'yang', '{', '}']



#tokenize
for intent in intents:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern.lower())  # Tokenizing and converting to lowercase
        words.extend([word for word in w if word not in ignore])  # Exclude words in ignore list
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dzikr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [14]:
# Melakukan stemming dan normalisasi data
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# Menghapus class duplikat dengan 'set'
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

735 documents
140 classes ['aktivitas', 'apa_itu_gangguan_mental', 'apa_kabar', 'apresiasi', 'balas_sapa', 'bantuan_darurat_bunuh_diri', 'berbicara_tentang_kesehatan_mental', 'berbicara_tentang_melukai_diri_sendiri', 'cara_memahami_diri_sendiri', 'cara_mengelola_emosi_negatif', 'dukungan_dan_kelompok', 'evaluasi_kegiatan', 'gejala_gangguan_kecemasan', 'haha', 'istirahat', 'kesulitan_emosional_ekstrem', 'lelucon', 'melukai_diri_sendiri', 'melukai_diri_sendiri_ekspresif', 'melukai_diri_sendiri_terhadap_keluarga', 'melukai_orang_lain', 'memahami_batasan_dalam_hubungan', 'memahami_kecemasan', 'memahami_pemicu_kecemasan', 'memahami_perasaan_terluka', 'memahami_stigma_terhadap_kesehatan_mental', 'membangun_habits_positif', 'membangun_hubungan_tersehat', 'membangun_hubungan_yang_sehat', 'membangun_kebiasaan_sehat_untuk_kesehatan_mental', 'membangun_kepercayaan_diri', 'membangun_kepercayaan_diri_dalam_pertemuan_sosial', 'membantu_seseorang_dengan_gangguan_mental', 'memperbaiki_kesehatan_mental

In [15]:
import random
import numpy as np

training = []
output_empty = [0] * len(classes)

for doc in documents:
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    bag = [1 if w in pattern_words else 0 for w in words]

    output_row = output_empty[:]
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])


In [10]:
# Reset the default TensorFlow graph
tf.compat.v1.reset_default_graph

# Define the model
model = keras.Sequential([
    layers.InputLayer(input_shape=(len(train_x[0]),), name='input_layer'),
    layers.Dense(128, activation='relu', name='hidden_layer1'),
    layers.Dense(64, activation='relu', name='hidden_layer2'),
    layers.Dense(len(train_y[0]), activation='softmax', name='output_layer')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up TensorBoard callback
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./logs')

# Train the model
history = model.fit(train_x, train_y, epochs=1000, batch_size=8, callbacks=[tensorboard_callback], verbose=1)

# Save the model
model.save('model.h5')

C:\Users\dzikr\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0094 - loss: 4.9450 
Epoch 2/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1426 - loss: 4.8026
Epoch 3/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2407 - loss: 4.3549
Epoch 4/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4456 - loss: 3.3771
Epoch 5/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6202 - loss: 2.3480
Epoch 6/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7053 - loss: 1.6523
Epoch 7/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7724 - loss: 1.1591
Epoch 8/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8445 - loss: 0.8054
Epoch 9/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8832 - loss: 0.6234
Epoch 10/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8819 - loss: 0.5358
Epoch 11/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9108 - loss: 0.4209
Epoch 12/1000
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/ste

In [58]:
import json

# Save
try:
    with open("training_data.json", "w") as file:
        # Convert non-serializable data like NumPy arrays or other objects to lists
        data = {'words': words, 'classes': classes, 'train_x': train_x.tolist(), 'train_y': train_y.tolist()}
        json.dump(data, file)
except IOError as e:
    print(f"Error saving training data: {e}")

In [16]:
# Save
try:
    with open("training_data", "wb") as file:
        pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, file)
except IOError as e:
    print(f"Error saving training data: {e}")

In [55]:
# Load
try:
    with open("training_data", "rb") as file:
        data = pickle.load(file)

    words = data['words']
    classes = data['classes']
    train_x = np.array(data['train_x'])
    train_y = np.array(data['train_y'])
except IOError as e:
    print(f"Error loading training data: {e}")

# Load model
try:
    model = keras.models.load_model('model.h5')
except IOError as e:
    print(f"Error loading model: {e}")


In [70]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
            if show_details:
                print(f"Found in bag: {w}")
    return bag

ERROR_THRESHOLD = 0.70



def classify(sentence):
    input_data = bow(sentence, words)
    results = model.predict(np.array([input_data]))[0]
    results = [[i, r] for i, r in enumerate(results) if r >= ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [(classes[r[0]], r[1]) for r in results]

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        for intent in intents:
            if intent['tag'] == results[0][0]:
                return random.choice(intent['responses'])
    return "Maaf, saya belum mengerti apa yang Anda bicarakan, perlu bantuan?."

In [71]:
clean_up_sentence("halo perkenalkan nama saya adalah")

['halo', 'perkenalkan', 'nama', 'saya', 'adalah']

In [21]:
print("0 to close")
while True:
    message = input("")
    if message == "0":
        break
    result = response(message)

    if result is not None and "~" in result:
        order = (result[1:])
        action(order)
    else:
        print(result)


0 to close


 hai


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Maaf, saya belum mengerti apa yang Anda bicarakan, perlu bantuan?.


 halo


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Halo di sana, ada yang bisa kubantu?


KeyboardInterrupt: Interrupted by user

In [ ]:
model = tf.keras.models.load_model('model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


with open('model.tflite', 'wb') as f:
  f.write(tflite_model)